In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score, accuracy_score
import tensorflow.keras as keras
import time
import numpy as np
import matplotlib.pyplot as plt
import math
from deep_audio import Directory, JSON
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import joblib

In [2]:
method_algo = 'mfcc'
model_algo = 'perceptron'
n_rate = 24000

DATASET_PATH = f'processed/{method_algo}/{method_algo}_{n_rate}.json'

inputs, targets, mapping = Directory.load_json_data(DATASET_PATH, inputs_fieldname=method_algo)

In [3]:
def build_model():
    # build the network architecture
    model = keras.Sequential([
        # input layer
        keras.layers.Flatten(input_shape=(inputs.shape[1], inputs.shape[2])),

        # 1st hidden layer
        keras.layers.Dense(512, activation='relu'),

        # 2nd hidden layer
        keras.layers.Dense(256, activation='relu'),

        # 3rd hidden layer
        keras.layers.Dense(128, activation='relu'),

        # output layer
        keras.layers.Dense(len(mapping), activation='softmax'),
    ])
    
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)

    model.compile(optimizer=optimizer,
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

    return model

In [4]:
# SPLIT DOS DADOS
random_state = 42


X_train, X_test, y_train, y_test = train_test_split(inputs,
                                                                          targets,
                                                                          test_size=0.2,
                                                                          stratify=targets,
                                                                          random_state=random_state)

X_train, X_valid, y_train, y_valid = train_test_split(X_train,
                                                                          y_train,
                                                                          test_size=0.2,
                                                                          stratify=y_train,
                                                                          random_state=random_state)


In [7]:
model = build_model()

# SALVA A ESTRUTURA DO MODELO

timestamp = int(time.time())

Directory.create_directory(f'models/{model_algo}/{method_algo}/{timestamp}')

JSON.create_json_file(f'models/{model_algo}/{method_algo}/{timestamp}/model_structure.json', model.to_json())

model_save_filename = f'models/{model_algo}/{method_algo}/{timestamp}/model_weight.h5'

# DECIDE QUANDO PARAR
earlystopping_cb = keras.callbacks.EarlyStopping(patience=300, restore_best_weights=True)

# SALVA OS PESOS
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
model_save_filename, monitor="val_accuracy", save_best_only=True
)

In [ ]:
# TREINA O MODELO
history = model.fit(X_train, y_train,
            validation_data=(X_valid, y_valid),
            epochs=10000,
            batch_size=128,
            callbacks=[earlystopping_cb, mdlcheckpoint_cb])

Epoch 1/10000
89/89 [==============================] - 4s 35ms/step - loss: 138.8102 - accuracy: 0.0130 - val_loss: 4.9647 - val_accuracy: 0.0138
Epoch 2/10000
89/89 [==============================] - 3s 31ms/step - loss: 4.8129 - accuracy: 0.0160 - val_loss: 4.7612 - val_accuracy: 0.0127
Epoch 3/10000
89/89 [==============================] - 2s 23ms/step - loss: 4.6974 - accuracy: 0.0129 - val_loss: 4.7429 - val_accuracy: 0.0127
Epoch 4/10000
89/89 [==============================] - 2s 21ms/step - loss: 4.6793 - accuracy: 0.0124 - val_loss: 4.7389 - val_accuracy: 0.0127
Epoch 5/10000
89/89 [==============================] - 2s 21ms/step - loss: 4.6738 - accuracy: 0.0113 - val_loss: 4.7375 - val_accuracy: 0.0127
Epoch 6/10000
89/89 [==============================] - 2s 21ms/step - loss: 4.6646 - accuracy: 0.0135 - val_loss: 4.7399 - val_accuracy: 0.0127
Epoch 7/10000
89/89 [==============================] - 2s 21ms/step - loss: 4.6689 - accuracy: 0.0153 - val_loss: 4.7412 - val_accurac

89/89 [==============================] - 2s 20ms/step - loss: 4.0133 - accuracy: 0.1114 - val_loss: 4.5883 - val_accuracy: 0.0771
Epoch 58/10000
89/89 [==============================] - 2s 21ms/step - loss: 4.0640 - accuracy: 0.0973 - val_loss: 4.4863 - val_accuracy: 0.0821
Epoch 59/10000
89/89 [==============================] - 2s 20ms/step - loss: 3.9873 - accuracy: 0.1109 - val_loss: 4.4729 - val_accuracy: 0.0831
Epoch 60/10000
89/89 [==============================] - 3s 31ms/step - loss: 3.9438 - accuracy: 0.1146 - val_loss: 4.3835 - val_accuracy: 0.0860
Epoch 61/10000
89/89 [==============================] - 2s 21ms/step - loss: 3.9146 - accuracy: 0.1295 - val_loss: 4.3907 - val_accuracy: 0.1033
Epoch 62/10000
89/89 [==============================] - 3s 32ms/step - loss: 3.8682 - accuracy: 0.1353 - val_loss: 4.5380 - val_accuracy: 0.1065
Epoch 63/10000
89/89 [==============================] - 2s 22ms/step - loss: 3.8488 - accuracy: 0.1441 - val_loss: 4.4829 - val_accuracy: 0.1128


89/89 [==============================] - 2s 21ms/step - loss: 1.3030 - accuracy: 0.6548 - val_loss: 3.7808 - val_accuracy: 0.4811
Epoch 114/10000
89/89 [==============================] - 2s 20ms/step - loss: 1.2727 - accuracy: 0.6637 - val_loss: 4.0418 - val_accuracy: 0.4963
Epoch 115/10000
89/89 [==============================] - 3s 31ms/step - loss: 1.1937 - accuracy: 0.6861 - val_loss: 3.8645 - val_accuracy: 0.4843
Epoch 116/10000
89/89 [==============================] - 2s 21ms/step - loss: 1.1234 - accuracy: 0.6866 - val_loss: 4.1954 - val_accuracy: 0.5023
Epoch 117/10000
89/89 [==============================] - 2s 21ms/step - loss: 1.0755 - accuracy: 0.7046 - val_loss: 3.6321 - val_accuracy: 0.5168
Epoch 118/10000
89/89 [==============================] - 2s 20ms/step - loss: 0.9848 - accuracy: 0.7228 - val_loss: 3.7694 - val_accuracy: 0.5235
Epoch 119/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.9745 - accuracy: 0.7286 - val_loss: 3.7548 - val_accuracy: 0

89/89 [==============================] - 2s 20ms/step - loss: 0.1595 - accuracy: 0.9672 - val_loss: 3.8685 - val_accuracy: 0.7075
Epoch 170/10000
89/89 [==============================] - 2s 20ms/step - loss: 0.1417 - accuracy: 0.9638 - val_loss: 3.8045 - val_accuracy: 0.7029
Epoch 171/10000
89/89 [==============================] - 2s 20ms/step - loss: 0.1275 - accuracy: 0.9701 - val_loss: 3.4283 - val_accuracy: 0.7184
Epoch 172/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.1095 - accuracy: 0.9758 - val_loss: 3.6677 - val_accuracy: 0.7163
Epoch 173/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.1208 - accuracy: 0.9740 - val_loss: 3.4649 - val_accuracy: 0.7121
Epoch 174/10000
89/89 [==============================] - 2s 20ms/step - loss: 0.1017 - accuracy: 0.9761 - val_loss: 3.7971 - val_accuracy: 0.7085
Epoch 175/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.1505 - accuracy: 0.9651 - val_loss: 4.0720 - val_accuracy: 0

89/89 [==============================] - 2s 21ms/step - loss: 0.0800 - accuracy: 0.9842 - val_loss: 3.8923 - val_accuracy: 0.7566
Epoch 226/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0722 - accuracy: 0.9850 - val_loss: 3.7266 - val_accuracy: 0.7520
Epoch 227/10000
89/89 [==============================] - 2s 20ms/step - loss: 0.0616 - accuracy: 0.9842 - val_loss: 3.6666 - val_accuracy: 0.7655
Epoch 228/10000
89/89 [==============================] - 2s 20ms/step - loss: 0.0724 - accuracy: 0.9835 - val_loss: 4.2617 - val_accuracy: 0.7478
Epoch 229/10000
89/89 [==============================] - 2s 20ms/step - loss: 0.0650 - accuracy: 0.9853 - val_loss: 3.7893 - val_accuracy: 0.7619
Epoch 230/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0646 - accuracy: 0.9868 - val_loss: 3.4770 - val_accuracy: 0.7747
Epoch 231/10000
89/89 [==============================] - 2s 20ms/step - loss: 0.0490 - accuracy: 0.9883 - val_loss: 3.5340 - val_accuracy: 0

89/89 [==============================] - 2s 21ms/step - loss: 0.0358 - accuracy: 0.9922 - val_loss: 3.8203 - val_accuracy: 0.7973
Epoch 282/10000
89/89 [==============================] - 2s 20ms/step - loss: 0.0602 - accuracy: 0.9884 - val_loss: 3.4861 - val_accuracy: 0.7920
Epoch 283/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0436 - accuracy: 0.9912 - val_loss: 3.3209 - val_accuracy: 0.8044
Epoch 284/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0232 - accuracy: 0.9938 - val_loss: 3.1659 - val_accuracy: 0.8185
Epoch 285/10000
89/89 [==============================] - 2s 20ms/step - loss: 0.0237 - accuracy: 0.9947 - val_loss: 3.4164 - val_accuracy: 0.8122
Epoch 286/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0141 - accuracy: 0.9966 - val_loss: 3.3499 - val_accuracy: 0.8090
Epoch 287/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0107 - accuracy: 0.9968 - val_loss: 3.2182 - val_accuracy: 0

89/89 [==============================] - 2s 20ms/step - loss: 0.0108 - accuracy: 0.9968 - val_loss: 3.4928 - val_accuracy: 0.8228
Epoch 338/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0180 - accuracy: 0.9962 - val_loss: 3.4971 - val_accuracy: 0.8136
Epoch 339/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0109 - accuracy: 0.9968 - val_loss: 3.6180 - val_accuracy: 0.8125
Epoch 340/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0141 - accuracy: 0.9953 - val_loss: 3.4773 - val_accuracy: 0.8100
Epoch 341/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0157 - accuracy: 0.9955 - val_loss: 3.3777 - val_accuracy: 0.8228
Epoch 342/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0138 - accuracy: 0.9964 - val_loss: 3.5675 - val_accuracy: 0.8185
Epoch 343/10000
89/89 [==============================] - 2s 20ms/step - loss: 0.0060 - accuracy: 0.9976 - val_loss: 3.2974 - val_accuracy: 0

89/89 [==============================] - 2s 21ms/step - loss: 0.0305 - accuracy: 0.9940 - val_loss: 4.0483 - val_accuracy: 0.8281
Epoch 394/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0150 - accuracy: 0.9966 - val_loss: 3.7262 - val_accuracy: 0.8401
Epoch 395/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0227 - accuracy: 0.9951 - val_loss: 4.7143 - val_accuracy: 0.8001
Epoch 396/10000
89/89 [==============================] - 2s 20ms/step - loss: 0.0606 - accuracy: 0.9917 - val_loss: 3.8704 - val_accuracy: 0.8369
Epoch 397/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0112 - accuracy: 0.9971 - val_loss: 3.5402 - val_accuracy: 0.8401
Epoch 398/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0235 - accuracy: 0.9959 - val_loss: 4.0910 - val_accuracy: 0.8362
Epoch 399/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0126 - accuracy: 0.9966 - val_loss: 3.8178 - val_accuracy: 0

89/89 [==============================] - 2s 21ms/step - loss: 0.0053 - accuracy: 0.9984 - val_loss: 3.4806 - val_accuracy: 0.8458
Epoch 450/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0077 - accuracy: 0.9977 - val_loss: 3.4820 - val_accuracy: 0.8458
Epoch 451/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0038 - accuracy: 0.9989 - val_loss: 3.4878 - val_accuracy: 0.8461
Epoch 452/10000
89/89 [==============================] - 2s 20ms/step - loss: 0.0040 - accuracy: 0.9989 - val_loss: 3.4928 - val_accuracy: 0.8465
Epoch 453/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0028 - accuracy: 0.9991 - val_loss: 3.4976 - val_accuracy: 0.8461
Epoch 454/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0065 - accuracy: 0.9977 - val_loss: 3.8339 - val_accuracy: 0.8291
Epoch 455/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0089 - accuracy: 0.9977 - val_loss: 3.5437 - val_accuracy: 0

In [ ]:
# GERA O GRAFICO DE ACURÁCIA
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_accuracy.png')
plt.close()

# GERA O GRÁFICO DE PERCA
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_loss.png')
plt.close()

In [ ]:
# PEGA A MAIOR E ACURÁCIA
higher_accuracy = model.evaluate(inputs_test, targets_test, batch_size=128)

higher_accuracy = str(int(higher_accuracy[1] * 10000)).zfill(4)

In [ ]:
# RENOMEIA A PASTA
Directory.rename_directory(f'models/{model_algo}/{method_algo}/{timestamp}',
                   f'models/{model_algo}/{method_algo}/acc{higher_accuracy}_seed{random_state}_epochs{len(history.history["accuracy"])}_time{timestamp}')